<a href="https://colab.research.google.com/github/UbaidullahTanoli/LLM-MLP/blob/main/LLM%2BMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade torch torchvision torchaudio

In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from torch.cuda.amp import GradScaler, autocast
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
    confusion_matrix
)
from sklearn.model_selection import KFold
from transformers import AutoTokenizer, AutoModel
from tqdm.auto import tqdm

In [3]:
class ReportDataset(Dataset):
    def __init__(self, reports_csv, max_length=256, tokenizer_name="dmis-lab/biobert-base-cased-v1.1"):
        """
        Args:
            reports_csv (str): Path to CSV file with medical reports
            max_length (int): Maximum sequence length for tokenization
            tokenizer_name (str): Pretrained tokenizer to use
        """
        # Load reports CSV
        self.reports_df = pd.read_csv(reports_csv)

        # Remove rows with empty text
        self.reports_df = self.reports_df[
            ~((self.reports_df['findings'].isna() | (self.reports_df['findings'].str.strip() == '')) &
              (self.reports_df['impression'].isna() | (self.reports_df['impression'].str.strip() == '')))
        ]

        # Initialize tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.max_length = max_length

        # Prepare labels
        self.labels = []
        for idx in range(len(self.reports_df)):
            row = self.reports_df.iloc[idx]
            mesh_val = str(row['MeSH']).strip().lower()
            label = 0 if mesh_val == 'normal' else 1
            self.labels.append(label)
        self.labels = np.array(self.labels)

    def __len__(self):
        return len(self.reports_df)

    def __getitem__(self, idx):
        row = self.reports_df.iloc[idx]

        # Concatenate relevant text fields
        report_text = ""
        if 'findings' in row and not pd.isna(row['findings']):
            report_text += "Findings: " + str(row['findings']) + " "
        if 'impression' in row and not pd.isna(row['impression']):
            report_text += "Impression: " + str(row['impression'])

        # Fallback if no text is found
        if not report_text.strip():
            report_text = "No report text available."

        # Tokenize the text
        tokenized = self.tokenizer(
            report_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # Extract and squeeze tensors (remove batch dimension)
        input_ids = tokenized['input_ids'].squeeze(0)
        attention_mask = tokenized['attention_mask'].squeeze(0)

        # Derive binary label from the "MeSH" column
        mesh_val = str(row['MeSH']).strip().lower()
        label = 0 if mesh_val == 'normal' else 1

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long),
            'uid': row['uid'] if 'uid' in row else -1
        }

    def get_class_distribution(self):
        """
        Returns the count of each class in the dataset.
        """
        return dict(zip(*np.unique(self.labels, return_counts=True)))

In [4]:
class ReportClassifier(nn.Module):
    def __init__(self, num_classes=2, freeze_bert=True, model_name="dmis-lab/biobert-base-cased-v1.1"):
        super(ReportClassifier, self).__init__()

        # Load pre-trained BioBERT
        self.bert = AutoModel.from_pretrained(model_name)

        # Freeze BERT parameters initially if specified
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

        # MLP classifier
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        # Get BioBERT embeddings
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # [CLS] token embedding
        #pooled_output = self.pooler_dropout(pooled_output)

        # Classification
        return self.classifier(pooled_output)

    def unfreeze_bert_layers(self, n_layers=1):
        """
        Unfreeze the last n transformer layers of BERT for fine-tuning
        """
        # Unfreeze the classifier
        for param in self.classifier.parameters():
            param.requires_grad = True

        # Unfreeze the last n BERT encoder layers
        for i in range(1, n_layers+1):
            layer_idx = self.bert.config.num_hidden_layers - i
            for param in self.bert.encoder.layer[layer_idx].parameters():
                param.requires_grad = True

In [5]:
def compute_metrics(labels, preds, probs):
    """
    Compute comprehensive metrics for model evaluation
    """
    # Basic metrics
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, zero_division=0)
    rec = recall_score(labels, preds, zero_division=0)
    f1 = f1_score(labels, preds, zero_division=0)

    # Confusion Matrix
    cm = confusion_matrix(labels, preds)
    tn, fp, fn, tp = cm.ravel()

    # Additional metrics
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # AUC metrics
    try:
        roc_auc = roc_auc_score(labels, probs)
        pr_auc = average_precision_score(labels, probs)
    except:
        roc_auc = 0.0
        pr_auc = 0.0

    return {
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'sensitivity': sensitivity,
        'specificity': specificity,
        'roc_auc': roc_auc,
        'pr_auc': pr_auc
    }

In [6]:
def train_epoch(model, dataloader, criterion, optimizer, device, scaler=None):
    model.train()
    running_loss = 0.0
    preds_all, labels_all = [], []
    probs_all = []

    # Initialize scaler if not provided
    if scaler is None:
        scaler = torch.amp.GradScaler(device='cuda')

    progress_bar = tqdm(dataloader, desc="Training", dynamic_ncols=True)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # Automatic Mixed Precision
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

        # Gradient scaling
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Clip gradients to norm 1.0
        scaler.step(optimizer)
        scaler.update()

        # Step the scheduler
        #scheduler.step()

        running_loss += loss.item() * input_ids.size(0)

        # Prediction and probabilities
        probs = torch.softmax(outputs, dim=1)[:, 1]
        _, preds = torch.max(outputs, dim=1)

        preds_all.extend(preds.cpu().numpy())
        labels_all.extend(labels.cpu().numpy())
        probs_all.extend(probs.detach().cpu().numpy())

        # Update progress bar
        progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    # Compute metrics
    epoch_loss = running_loss / len(dataloader.dataset)
    metrics = compute_metrics(labels_all, preds_all, probs_all)
    metrics['loss'] = epoch_loss

    return metrics

In [7]:
def eval_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    preds_all, labels_all = [], []
    probs_all = []

    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Evaluating", dynamic_ncols=True)
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Automatic Mixed Precision
            with torch.amp.autocast(device_type='cuda'):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = criterion(outputs, labels)

            running_loss += loss.item() * input_ids.size(0)

            # Prediction and probabilities
            probs = torch.softmax(outputs, dim=1)[:, 1]
            _, preds = torch.max(outputs, dim=1)

            preds_all.extend(preds.cpu().numpy())
            labels_all.extend(labels.cpu().numpy())
            probs_all.extend(probs.detach().cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    # Compute metrics
    epoch_loss = running_loss / len(dataloader.dataset)
    metrics = compute_metrics(labels_all, preds_all, probs_all)
    metrics['loss'] = epoch_loss

    return metrics

In [8]:
def train_and_evaluate(reports_csv, num_epochs=40, batch_size=8, learning_rate=2e-4, test_size=0.2, random_state=42):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load the full dataset
    full_dataset = ReportDataset(reports_csv)

    # Check class distribution
    class_distribution = full_dataset.get_class_distribution()
    print(f"Class distribution: {class_distribution}")

    # Best model tracking
    best_test_metrics = {'f1': 0}

    # Split the dataset into train and test sets
    train_size = int((1 - test_size) * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(
        full_dataset,
        [train_size, test_size],
        generator=torch.Generator().manual_seed(random_state)
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )

    # Calculate class weights
    num_samples = len(train_dataset)
    class_weights = torch.FloatTensor([
        num_samples / (len(class_distribution) * count)
        for count in class_distribution.values()
    ]).to(device)

    # Initialize model
    model = ReportClassifier(num_classes=2, freeze_bert=True).to(device)

    # Loss function with class weights
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # First phase: Train only the classifier head
    print("Phase 1: Training only the classifier head...")
    first_phase_params = model.classifier.parameters()
    #first_phase_optimizer = optim.AdamW(first_phase_params, lr=1e-3, weight_decay=0.01)
    #first_phase_scheduler = optim.lr_scheduler.LinearLR(first_phase_optimizer)
    #scaler = torch.amp.GradScaler(device='cuda')

    # Calculate total training steps
    #total_steps = len(train_loader) * (num_epochs)
    #scheduler = get_linear_schedule_with_warmup(
    #    first_phase_optimizer,
    #    num_warmup_steps=int(0.1 * total_steps),
    #    num_training_steps=total_steps
    #    )

    first_phase_optimizer = optim.AdamW(first_phase_params, lr=1e-3, weight_decay=0.01)

    first_phase_scheduler = optim.lr_scheduler.LinearLR(
    first_phase_optimizer,
    start_factor=1.0,
    end_factor=0.1,
    total_iters=num_epochs
    )

    scaler = torch.amp.GradScaler(device='cuda')


    # Train classifier head
    for epoch in range(1, 41):
        train_metrics = train_epoch(
            model,
            train_loader,
            criterion,
            first_phase_optimizer,
            device, scaler
        )
        val_metrics = eval_epoch(model, test_loader, criterion, device)

        # In training loop (for each epoch), after computing metrics:
        print(f"Epoch {epoch}/{num_epochs} - "
              f"Train Loss: {train_metrics['loss']:.4f} | "
              f"Acc: {train_metrics['accuracy']:.4f} | "
              f"Prec: {train_metrics['precision']:.4f} | "
              f"Rec: {train_metrics['recall']:.4f} | "
              f"F1: {train_metrics['f1']:.4f} | "
              f"Sens: {train_metrics['sensitivity']:.4f} | "
              f"Spec: {train_metrics['specificity']:.4f} | "
              f"ROC AUC: {train_metrics['roc_auc']:.4f} | "
              f"PR AUC: {train_metrics['pr_auc']:.4f}")

        print(f"                Val Loss: {val_metrics['loss']:.4f} | "
              f"Acc: {val_metrics['accuracy']:.4f} | "
              f"Prec: {val_metrics['precision']:.4f} | "
              f"Rec: {val_metrics['recall']:.4f} | "
              f"F1: {val_metrics['f1']:.4f} | "
              f"Sens: {val_metrics['sensitivity']:.4f} | "
              f"Spec: {val_metrics['specificity']:.4f} | "
              f"ROC AUC: {val_metrics['roc_auc']:.4f} | "
              f"PR AUC: {val_metrics['pr_auc']:.4f}")

        # Update the learning rate for the next epoch
        first_phase_scheduler.step()


        # Save best model based on F1 score
        if val_metrics['f1'] > best_test_metrics['f1']:
            best_test_metrics = val_metrics
            torch.save(model.state_dict(), "best_biobert_model.pt")
            print(f"  Saved new best model with test F1: {val_metrics['f1']:.4f}")

    print("\nFinal Test Results:")
    for metric, value in best_test_metrics.items():
        print(f"  {metric}: {value:.4f}")

    return model, best_test_metrics

In [9]:
def main():
    # Path to the CSV file
    reports_csv = "/content/indiana_reports.csv"

    # Run training and evaluation
    model, metrics = train_and_evaluate(reports_csv, num_epochs=40, batch_size=8)

In [10]:
if __name__ == "__main__":
    main()

Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

Class distribution: {np.int64(0): np.int64(1354), np.int64(1): np.int64(2472)}


...

Evaluating:   0%|          | 0/96 [00:00<?, ?it/s]

Epoch 39/40 - Train Loss: 0.3604 | Acc: 0.8703 | Prec: 0.9027 | Rec: 0.8968 | F1: 0.8998 | Sens: 0.8968 | Spec: 0.8211 | ROC AUC: 0.9367 | PR AUC: 0.9641
                Val Loss: 0.2657 | Acc: 0.8864 | Prec: 0.9716 | Rec: 0.8454 | F1: 0.9041 | Sens: 0.8454 | Spec: 0.9573 | ROC AUC: 0.9720 | PR AUC: 0.9824


Training:   0%|          | 0/383 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/96 [00:00<?, ?it/s]

Epoch 40/40 - Train Loss: 0.3664 | Acc: 0.8703 | Prec: 0.9064 | Rec: 0.8923 | F1: 0.8993 | Sens: 0.8923 | Spec: 0.8295 | ROC AUC: 0.9361 | PR AUC: 0.9635
                Val Loss: 0.2646 | Acc: 0.8903 | Prec: 0.9696 | Rec: 0.8536 | F1: 0.9079 | Sens: 0.8536 | Spec: 0.9537 | ROC AUC: 0.9729 | PR AUC: 0.9828

Final Test Results:
  accuracy: 0.9321
  precision: 0.9577
  recall: 0.9340
  f1: 0.9457
  sensitivity: 0.9340
  specificity: 0.9288
  roc_auc: 0.9658
  pr_auc: 0.9781
  loss: 0.2384
